# Maskless photolitography with Micro-Manager

This workflow is designed for microscopes that can be controlled with uManager drivers (see https://micro-manager.org/Device_Support for a list of all compatible devices).

Required hardware device:
- Any DMD/SLM capable of projecting images with UV light ([GenericSLM](https://micro-manager.org/GenericSLM))
- Light source with UV and another wavelength that does not cure the resin
- XY stage to use as stepper (XYStageDevice)
- Camera to find focus plane and align layers for 2.5D printing

In [1]:
from manager.acquisition import acq, acq_mask
from manager.dmd import dmd
from manager.fov import FOV
from manager.preset import preset
from manager.stage import stage_position
from utils.fabscope_ui import FabscopeUI
    
from matplotlib import pyplot as plt 
from pycromanager import Bridge
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np

matplotlib.rcParams["image.interpolation"] = None
nest_asyncio.apply()

Validation errors in config file(s).
The following fields have been reset to the default value:

schema_version
  value is not a valid tuple (type=type_error.tuple)



## Setup connection between Pycro-Manager and Micro-Manager

In [2]:
bridge = Bridge() 
core = bridge.get_core()
studio = bridge.get_studio()
print(bridge.get_core())
dmd = dmd(core) #init dmd device

### Set up basic microscope config (light path, hardware triggering, binning etc)

In [3]:
nidaq_setup = preset(core) #focus preset
nidaq_setup.settings = [
    ["Spectra RIGHT","Teal_Level",100],
    ["Spectra RIGHT","Teal_Enable",0],
    ["Spectra RIGHT","Cyan_Level",100],
    ["Spectra RIGHT","Cyan_Enable",0],
    ["Spectra RIGHT","Violet_Level",100],
    ["Spectra RIGHT","Violet_Enable",0],
    ["Spectra RIGHT","Green_Level",100],
    ["Spectra RIGHT","Green_Enable",0],
    ["Spectra RIGHT","Blue_Level",100],
    ["Spectra RIGHT","Blue_Enable",0],
    ["Spectra RIGHT","Red_Level",100],
    ["Spectra RIGHT","Red_Enable",0],
    ["Spectra RIGHT","White_Enable",0],
    ['Andor sCMOS LEFT','AuxiliaryOutSource (TTL I/O)','FireAny']
]
nidaq_setup.apply_no_retry()

camera_setup = preset(core) #focus preset
camera_setup.settings = [
    ["Andor sCMOS LEFT","Binning",'2x2'],
    ["TILightPath","State",'2'],
    ["Mosaic3","TriggerMode",'InternalExpose'],
]
camera_setup.apply(core)

Andor sCMOS LEFT: Set Binning to 2x2
TILightPath: Set State to 2
Mosaic3: Set TriggerMode to InternalExpose


### Hardware setup for UV exposure
The channel/light source you want to activate when doing a UV exposure of the resin

In [4]:
expose_UV = preset(core)
expose_UV.settings = [
    ["Spectra RIGHT","Teal_Enable",0],
    ["Spectra RIGHT","Cyan_Enable",0],
    ["Spectra RIGHT","Violet_Enable",1],
    ["Spectra RIGHT","Blue_Enable",0],
    ["Spectra RIGHT","Red_Enable",0],
    ["TIFilterBlock1", "State", "1"],#2 
    ["Wheel-C", "State", 1],
    ["Spectra RIGHT","Green_Enable",0],
    ["Spectra RIGHT","Violet_Level",100]
]
expose_UV.camera_exposure_time = 250 
expose_UV.name = 'expose_UV'
expose_UV.apply(core)

Spectra RIGHT: Set Teal_Enable to 0
Spectra RIGHT: Set Cyan_Enable to 0
Spectra RIGHT: Set Violet_Enable to 1
Spectra RIGHT: Set Blue_Enable to 0
Spectra RIGHT: Set Red_Enable to 0
TIFilterBlock1: Set State to 1
Wheel-C: Set State to 1
Spectra RIGHT: Set Green_Enable to 0
Spectra RIGHT: Set Violet_Level to 100


### Hardware setup for focus checking
A light source that does not cure the resin, in our case a red LED.

In [8]:
check_RED = preset(core)
check_RED.settings = [ 
    ["TIFilterBlock1", "State", "1"],
    ["Wheel-C", "State", 2], 
    ["Spectra RIGHT","Teal_Enable",0],
    ["Spectra RIGHT","Cyan_Enable",0],
    ["Spectra RIGHT","Violet_Enable",0],
    ["Spectra RIGHT","Green_Enable",0],
    ["Spectra RIGHT","Blue_Enable",0],
    ["Spectra RIGHT","Red_Enable",1],
    ["Spectra RIGHT","Red_Level",50] 
]
check_RED.camera_exposure_time = 5 
check_RED.dmd_exposure_time = 250
check_RED.name = "check_RED"
check_RED.apply()

### Hardware settings to turn off all lights

In [10]:
dark = preset(core)
dark.settings = [
    ["Spectra RIGHT","Teal_Enable",0],
    ["Spectra RIGHT","Cyan_Enable",0],
    ["Spectra RIGHT","Violet_Enable",0],
    ["Spectra RIGHT","Green_Enable",0],
    ["Spectra RIGHT","Blue_Enable",0],
    ["Spectra RIGHT","Red_Enable",0],
    ["Wheel-C", "State", 0],
]
dark.name = 'BLACK'
dark.apply()

### Add all channels to the list that should show up in the user interface

In [11]:
channels = [expose_UV,check_RED,dark]

## Start the napari GUI for liveview

In [13]:
viewer = FabscopeUI(
    core=core,
    dmd=dmd,
    channels=channels,
    simulate=False,
    sleep_time=0.1,
    clim=(0, 255)
)

# Initialize and show the viewer
viewer.initialize_viewer()

viewer already closed or never opened


check_RED
stopping threads
starting threads...
Worker started: yield_img
Worker started: append_img
stopping threads
acquisition done
starting threads...
Worker started: yield_img
Worker started: append_img
stopping threads
acquisition done


### New calibration of the X/Y step size
How much should the stage move between FOVs?
To calibrate, first 
1. Make a full FOV exposure with UV to cure the resin (`position_1`)
2. Turn on the other light source that does note cure the resin
3. Move the stage to align the so that the current top left corner precisely touches the bottom right corner of the printed structure (`position_2`)

By calculating the vector between the two positions, we get the x/y translation needed.

For our 20x objective, the values are:
`x_offset = 660.7`
`y_offset = 482.5`

In [ ]:
# epose the whole FOV and store the first position
mask = np.ones((600,800)).astype('uint8')
stim_img = acq_mask(mask, expose_UV, dmd)
position_1 = core.get_xy_stage_position()

now move to position 2

In [27]:
# store the second position
position_2 = core.get_xy_stage_position()

x_offset = position_1.get_x()-position_2.get_x()
y_offset = position_1.get_y()-position_2.get_y()

print(x_offset)
print(y_offset)

-6114.20009110868
0.0


### Use a previously measured X/Y step size (or adapt it to a different objective)

In [ ]:
calibration_objective = 20 #objective that was used for calibration
current_objective = 20 #objective that is currently used

#measured offset
y_offset = 482.5 
x_offset = 660.7

y_offset = calibration_objective/current_objective*485
x_offset = calibration_objective/current_objective*655

### Load a large image mask and tile it

In [ ]:
from utils.mask_handler import load_mask, tile_array, visualize_tiles

# Define the base path
base_path = 'Z:\\PertzLab\\lhinder\\micropatterning'

# Load a mask
mask = load_mask(base_path, 'your_mask_file.tif', invert=False)

# Split into tiles with custom dimensions if needed
tiles, grid_width, grid_height = tile_array(mask, tile_width=800, tile_height=600)

# Optionally visualize the tiles
visualize_tiles(tiles, grid_width, grid_height)
print(f'Number of tiles: {len(tiles)}, {grid_width}x{grid_height} (rows x cols)')

### Expose the mask, stepping through all tiles

In [ ]:
exposure_calibration = False #produce a exposure calibration grid
test_mode = False #use red LED instead of UV, to check if the stage is moving as expected

#store the led power setting and exposure time for the UV LED you want to test in an array
led_powers = np.linspace(10,100,grid_width).astype(int)
exposure_times = np.linspace(10,1,grid_height).astype(int)

#check if led_powers and exposure_times are same lenght as grid_width and grid_height
if len(led_powers) != grid_width:
    print('led_powers and grid_width are not the same length')
if len(exposure_times) != grid_height:
    print('exposure_times and grid_height are not the same length')

### Run!

In [ ]:
point = core.get_xy_stage_position()
x_pos = point.get_x()
y_pos = point.get_y()

pos = stage_position(x_pos,y_pos,None)
fov_start = FOV(core, pos, 'treatment', 2,  10, 10,10)
fov = fov_start

exposure_preset = expose_UV if not test_mode else check_RED

stim_imgs = []
with tqdm(total=grid_width*grid_height) as pbar:
    for col in range(grid_width):
        for row in range(grid_height):
            y_pos_new = y_pos+(row*y_offset)
            x_pos_new = x_pos+(col*x_offset)
            pos = stage_position(x_pos_new,y_pos_new,None)
            fov = FOV(core, pos, 'treatment', 2, 10, 10, 10)
            fov.move_stage_to_fov()

            if exposure_calibration:
                exposure_preset.camera_exposure_time = exposure_times[row]
                exposure_preset.set_power(led_powers[col])
                exposure_preset.apply()

            stim_img = acq_mask(tiles[row][col], exposure_preset, dmd)
            stim_imgs.append(stim_img)
            
            pbar.update(1)
            
fov_start.move_stage_to_fov()

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [01:51<00:00,  1.11s/it]


#### QC of the stimulation images
When exposing the layer with UV light, we also acquire the image. This can be useful for trouble shooting, e.g. detecting impurities or focus drift.

In [ ]:
plt.imshow(stim_imgs[0])
plt.show()